# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [1]:
from collections import defaultdict
import glob
import os
from typing import Tuple

import numpy as np
import xarray as xr
from e3sm_diags.derivations.derivations import DERIVED_VARIABLES


DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-dev/**"

DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc", recursive=True))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc", recursive=True))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [3]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")

In [4]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    results = {
        "missing_files": [],
        "matching_files": [],
        "mismatch_errors": [],
        "not_equal_errors": [],
        "key_errors": [],
    }

    for fp_main in MAIN_GLOB:
        if "_diff.nc" not in fp_main:
            fp_dev = fp_main.replace("examples-main", "examples-dev")

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            try:
                ds1 = xr.open_dataset(fp_dev)
                ds2 = xr.open_dataset(fp_main)
            except FileNotFoundError as e:
                print(f"    {e}")

                if isinstance(e, FileNotFoundError):
                    results["missing_files"].append(fp_dev)

                continue

            var_key = fp_main.split("-")[-3]
            # for 3d vars such as T-200
            var_key.isdigit()
            if var_key.isdigit():
                var_key = fp_main.split("-")[-4]

            print(f"    * var_key: {var_key}")

            dev_data = _get_var_data(ds1, var_key)
            main_data = _get_var_data(ds2, var_key)

            if dev_data is None or main_data is None:
                print("    * Could not find variable key in the dataset(s)")
                continue

            try:
                np.testing.assert_allclose(
                    dev_data,
                    main_data,
                    atol=atol,
                    rtol=rtol,
                )
                results["matching_files"].append(fp_main)
            except (KeyError, AssertionError) as e:
                msg = str(e)

                print(f"    {msg}")
                if "mismatch" in msg:
                    results["mismatch_errors"].append(fp_main)
                elif "Not equal to tolerance" in msg:
                    results["not_equal_errors"].append(fp_main)
            else:
                print(f"    * All close and within relative tolerance ({rtol})")

    return results


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    var_keys = DERIVED_VARIABLES[var_key].keys()
    var_keys = [var_key] + list(sum(var_keys, ()))

    for key in var_keys:
        if key in ds.data_vars.keys():
            data = ds[key].values
            break

    return data

## 1. Check for matching and equal number of files


In [5]:
_check_if_files_found()

In [6]:
_check_if_matching_filecount()

OSError: Number of files do not match at DEV_PATH and MAIN_PATH (4677 vs. 3091).

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [6]:
results = _get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-dev/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model/-ERFtot-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model/-ERFtot-ANN-global_test.nc
    * var_key: ERFtot
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-dev/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model/-FLNS-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model/-FLNS-ANN-global_test.nc
    * var_key: FLNS
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-dev/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model/-FLUT-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_m

In [7]:
(
    missing_files,
    matching_files,
    mismatch_errors,
    not_equal_errors,
    key_errors,
) = results.values()

In [8]:
sum_files_compared = (
    len(matching_files)
    + len(mismatch_errors)
    + len(not_equal_errors)
    + len(key_errors)
    + len(missing_files)
)

### Close results


In [9]:
pct_match = (len(matching_files) / sum_files_compared) * 100

print(
    f"Matching files count: {len(matching_files)}/{sum_files_compared}, {pct_match:.2f}%"
)

Matching files count: 1993/2078, 95.91%


### Missing Files


In [10]:
pct_missing = (len(missing_files) / sum_files_compared) * 100

print(f"Missing files count: {len(missing_files)}, {pct_missing:.2f}%")

Missing files count: 0, 0.00%


### `NaN` Mismatching Errors (Not concerned)

I found these `nan` mismatch errors occur due to either:

1. Regional subsetting on "ccb" flag in CDAT adding a coordinate points -- removing these coordinates results in matching results
2. Slightly different masking in the data between xCDAT and CDAT via xESMF/ESMF -- same number of nans just slightly shifted over some coordinates points

- Refer to PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)


In [12]:
pct_mismatch = (len(mismatch_errors) / sum_files_compared) * 100

print(f"Mismatch errors count: {len(mismatch_errors)}, {pct_missing:.2f}%")

Mismatch errors count: 61, 0.00%


In [13]:
mismatch_errors

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex2_modTS_vs_modTS_CMIP_3years/lat_lon/model_vs_model/-TAUXY-ANN-ocean_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex2_modTS_vs_modTS_CMIP_3years/lat_lon/model_vs_model/-TREFHT-ANN-land_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex2_modTS_vs_modTS_CMIP_3years/lat_lon/model_vs_model/-TREFMNAV-ANN-land_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex2_modTS_vs_modTS_CMIP_3years/lat_lon/model_vs_model/-TREFMXAV-ANN-land_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex2_modTS_vs_modTS_CMIP_3years/lat_lon/model_vs_model/-TREF_range-ANN-land_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex3_modTS_CMIP_vs_obs_2years/lat_lon/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANN-75S75N_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex3_modTS_CMIP_vs_obs_2years/lat_lon/CRU_I

### Not Equal Errors (Not concerned)

Refer to GH PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)

> The following variables have large diffs due to a bug on main: MISRCOSP-CLDLOW_TAU1.3_9.4_MISR, MISRCOSP-CLDLOW_TAU1.3_MISR, MISRCOSP-CLDLOW_TAU9.4_MISR. I believe this branch is doing the correct thing. We can revisit the comparison for these variables once the issue below is fixed on main.
>
> [\[Bug\]: Silent bug in adjust_prs_val_units() conditional where if prs_val0: will be False if prs_val0 is 0 #797](https://github.com/E3SM-Project/e3sm_diags/issues/797)


In [14]:
pct_not_equal_errors = (len(not_equal_errors) / sum_files_compared) * 100

print(
    f"Not equal to tolerance count: {len(not_equal_errors)}, {pct_not_equal_errors:.2f}%"
)

Not equal to tolerance count: 24, 1.15%


In [15]:
not_equal_errors

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud ISCCP/ISCCPCOSP-CLDTOT_TAU1.3_9.4_ISCCP-ANN-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud ISCCP/ISCCPCOSP-CLDTOT_TAU1.3_ISCCP-ANN-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud ISCCP/ISCCPCOSP-CLDTOT_TAU9.4_ISCCP-ANN-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud MISR/MISRCOSP-CLDLOW_TAU1.3_9.4_MISR-ANN-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud MISR/MISRCOSP-CLDLOW_TAU1.3_9.4_MISR-DJF-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud MISR/MISRCOSP-CLDLOW_TAU1.3_9.4_MISR-JJA-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/examples-main/ex5_model_to_obs/lat_lon/Cloud

### Key Errors (None)


In [13]:
print(f"Key errors count: {len(key_errors)}")

Key errors count: 0


## Results

- After testing all of the examples, we should be good to go here.
